# 1 关系型数据库功能验证

## 1.1 用户登陆鉴权验证

In [1]:
# 确保当前notebook可以正常导入models相关模块
import sys

sys.path.append('..')


In [2]:

import hashlib
# 注：这里由于是在notebooks/数据模型测试.ipynb中执行代码
# 因此针对数据库模型所生成的SQLite对应db文件与当前ipynb文件同级
# 好处是不会与基于项目根目录启动后创建的db文件相冲突
from models.sql import sql_db, UserInfo


In [3]:
def str2md5(raw_str: str) -> str:
    """对字符串进行md5加密

    Args:
        raw_str (str): 输入的待加密字符串

    Returns:
        str: 加密后的md5值
    """

    m = hashlib.md5()
    m.update(raw_str.encode('utf-8'))

    return m.hexdigest()


In [4]:

# 向用户信息表中插入单条测试用模拟数据
with sql_db.atomic():

    # 若测试表中无下列测试数据，则进行插入
    if UserInfo.select(UserInfo.user_id == 'test').count() == 0:
        UserInfo.create(
            user_id='test',
            user_name='张三',
            password='test_password',
            md5_password=str2md5('test_password'),
            gender='男',
            department_id=1,
            rank_id=1
        )

# 查询插入的测试数据
list(UserInfo.select().where(UserInfo.user_id == 'test').dicts())


[{'user_id': 'test',
  'user_name': '张三',
  'password': 'test_password',
  'md5_password': '16ec1ebb01fe02ded9b7d5447d3dfc65',
  'gender': '男',
  'department_id': 1,
  'rank_id': 1}]

In [5]:
# 模拟用户登陆鉴权验证，这里将具有唯一性的user_id视作登陆用户名

def check_user_auth(user_id: str,
                    md5_password: str) -> dict:

    # 尝试根据输入的user_id查询用户信息记录
    with sql_db.atomic():
        match_records = (
            UserInfo
            .select()
            .where(UserInfo.user_id == user_id)
            .dicts()
        )

    # 判断当前用户是否存在
    if match_records:
        # 继续判断md5密码是否正确
        if match_records[0]['md5_password'] == md5_password:
            return {
                'status': 'success',
                'message': '当前用户鉴权通过',
                'data': match_records[0]
            }

        return {
            'status': 'error',
            'message': '当前用户密码错误'
        }

    return {
        'status': 'error',
        'message': '当前用户不存在'
    }


In [6]:
test_input_params = [
    # 模拟用户id存在且md5密码正确的情况
    ('test', str2md5('test_password')),
    # 模拟用户id存在但md5密码错误的情况
    ('test', str2md5('test_password_wrong')),
    # 模拟用户id不存在的情况
    ('test_wrong', 'demo_password'),
]

In [7]:
check_user_auth(*test_input_params[0])


{'status': 'success',
 'message': '当前用户鉴权通过',
 'data': {'user_id': 'test',
  'user_name': '张三',
  'password': 'test_password',
  'md5_password': '16ec1ebb01fe02ded9b7d5447d3dfc65',
  'gender': '男',
  'department_id': 1,
  'rank_id': 1}}

In [8]:
check_user_auth(*test_input_params[1])


{'status': 'error', 'message': '当前用户密码错误'}

In [9]:
check_user_auth(*test_input_params[2])


{'status': 'error', 'message': '当前用户不存在'}

# 2 非关系型数据库功能验证

## 2.1 流程元信息表数据插入验证

In [10]:
# 确保当前notebook可以正常导入models相关模块
import sys

sys.path.append('..')

In [11]:
from models.nosql import ProcessMeta, init_db


In [12]:
# 强制重置数据库
init_db(init_process_meta=True)


集合process_meta重置成功


In [13]:
# 插入测试数据
process_meta = ProcessMeta()

(
    process_meta
    .collection
    .insert_one(
        {
            '流程id': 'test_process_id',
            '流程名称': '测试流程1',
            '流程描述': '这是测试流程1',
            '流程类型': '测试类型1',
            '可发起部门': ['部门A', '部门B'],
            '流程表单结构': [
                {
                    '字段名': '字段1',
                    '字段描述': '这是字段1',
                    '类型': '字符型',
                    '是否必填': True
                },
                {
                    '字段名': '字段2',
                    '字段描述': '这是字段2',
                    '类型': '字符型',
                    '是否必填': True,
                    '约束条件': {
                        '约束类型': '枚举约束',
                        '枚举范围': ['a', 'b', 'c', 'd']
                    }
                },
                {
                    '字段名': '字段3',
                    '字段描述': '这是字段3',
                    '类型': '整型',
                    '是否必填': True,
                    '约束条件': {
                        '约束类型': '范围约束',
                        '下限': 0
                    }
                }
            ],
            '审批步骤': [
                {
                    '步骤名称': '测试步骤1',
                    '步骤描述': '这是测试步骤1',
                    '步骤次序': 1,
                    '是否允许跳过': False,
                    '是否为末尾节点': True,
                    '审批人规则': {
                        '规则类型': '相对职级审批人',
                        '规则目标': '部门正职'
                    }
                }
            ]
        }
    )
)


In [14]:
(
    process_meta
    .collection
    .find_one(
        {
            '流程id': 'test_process_id'
        }
    )
)


{'_id': ObjectId('6424486fc045c6e94d02eca5'),
 '流程id': 'test_process_id',
 '流程名称': '测试流程1',
 '流程描述': '这是测试流程1',
 '流程类型': '测试类型1',
 '可发起部门': ['部门A', '部门B'],
 '流程表单结构': [{'字段名': '字段1', '字段描述': '这是字段1', '类型': '字符型', '是否必填': True},
  {'字段名': '字段2',
   '字段描述': '这是字段2',
   '类型': '字符型',
   '是否必填': True,
   '约束条件': {'约束类型': '枚举约束', '枚举范围': ['a', 'b', 'c', 'd']}},
  {'字段名': '字段3',
   '字段描述': '这是字段3',
   '类型': '整型',
   '是否必填': True,
   '约束条件': {'约束类型': '范围约束', '下限': 0}}],
 '审批步骤': [{'步骤名称': '测试步骤1',
   '步骤描述': '这是测试步骤1',
   '步骤次序': 1,
   '是否允许跳过': False,
   '是否为末尾节点': True,
   '审批人规则': {'规则类型': '相对职级审批人', '规则目标': '部门正职'}}]}